In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
# Input ranked list filename
in_ranked_list_filename = "ranking_list_rf.csv"
#in_ranked_list_filename = "ranking_list_LM.csv"
#in_ranked_list_filename = "ranking_list_baseline.csv"
#in_ranked_list_filename = "ranking_list_GBM.csv"

In [3]:
# Save results to this file
out_nDCGs_filename = "out_nDCGs_rf.txt"
#out_nDCGs_filename = "out_nDCGs_LM.txt"
#out_nDCGs_filename = "out_nDCGs_baseline.txt"
#out_nDCGs_filename = "out_nDCGs_GBM.txt"

In [4]:
# filename of optimal DCGs and ground truth
opt_DCGs_filename = "optimal_DCGs.txt"
ground_truth_filename = "test1.csv"

In [5]:
# Load optimal DCGs
opt_DCGs = {}
with open(opt_DCGs_filename,'r') as f:
    for line in f:
        x = line.strip('\n').split(' ')
        opt_DCGs[x[0]] = float(x[1])

In [6]:
opt_DCGs;

In [7]:
# Load ranked list
df_RL = pd.read_csv(in_ranked_list_filename, sep=',')

In [8]:
df_RL

,srch_id,prop_id
0,266262,40200
1,266262,24575
2,266262,35788
3,266262,37381
4,266262,37671
5,266262,50431
6,266262,51644
7,266262,59132
8,266262,65022
9,266262,65147


In [9]:
# Load ground truth
df_test = pd.read_csv(ground_truth_filename, sep=',')

In [10]:
df_test = df_test[['srch_id','prop_id','click_bool','booking_bool']]

In [11]:
df_test;

In [12]:
all_qids = list(df_test['srch_id'].unique())
all_qids;

In [13]:
len(all_qids)

40012

In [14]:
x = all_qids[1]
opt_DCGs[str(x)]

31.0

In [15]:
out_nDCGs_f = open(out_nDCGs_filename, 'w')

In [16]:
qw=0
for qid in all_qids:
    qw=qw+1
    if qw % 1000 == 0:
        print(qw)
    df_cur_r = df_RL[df_RL['srch_id'] == qid]
    df_cur_t = df_test[df_test['srch_id'] == qid]
    
    if df_cur_r.shape[0] != df_cur_t.shape[0]:
        print("qqqq")
        break
    
    # Compute relevance scores for each prop using df_cur_t
    rel_scores = {}
    for i,row in df_cur_t.iterrows():
        if row['booking_bool'] == 1:
            rel_scores[row['prop_id']] = 5
        elif row['click_bool'] == 1:
            rel_scores[row['prop_id']] = 1
        else:
            rel_scores[row['prop_id']] = 0
    
    # Compute DCG of current qid based on df_cur_r
    pos = 1
    DCG = 0
    for i,row in df_cur_r.iterrows():
        rel_score = rel_scores[row['prop_id']]
        DCG = DCG + (2**rel_score -1) / math.log2(pos+1)
        pos = pos + 1
    
    # Compute nDCG
    opt_DCG = opt_DCGs[str(qid)]
    nDCG = DCG / opt_DCG

    out_nDCGs_f.write(str(qid) + "," + str(nDCG) + '\n')

out_nDCGs_f.close()
print(qw)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
40012
